In [ ]:
from utilities import *

In [ ]:
############### Functions to get and generated paid months ################
MONTHS = ["NA", "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
          "JULIO", "AGOSTO", "SETIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"]
SMONTHS = ["NA", "ENE", "FEB", "MAR", "ABR", "MAY", "JUN", "JUL", "AGO", "SET", "OCT", "NOV", "DIC"]

def get_months(months_list, year, get_range=False):    
    imonths = []
    for m in months_list:
        if m and not m.isspace():
            m = m.replace(' ','').replace(',','')
            if m in MONTHS:
                imonths.append(MONTHS.index(m))
            elif m in SMONTHS:
                imonths.append(SMONTHS.index(m))
            else:
                #print("Warn: month not identified:" + m + ".")
                continue
            
    if get_range and len(imonths) == 2:
        imonths = list(range(imonths[0], imonths[1]+1))
    
    out = []
    for m in imonths:
        out.append([year, m])
    return out

def gen_month_description(y, mlist):
    if len(mlist) > 1:
        return "DE {} A {} {}".format(MONTHS[mlist[0]],MONTHS[mlist[-1]],y)
    else:
        return "{} {}".format(MONTHS[mlist[0]],y)
    
def gen_next_description(lastmonth, nmonths=1):
    y = int(lastmonth[0])
    m = int(lastmonth[1])
    d = []
    
    mlist = list()
    for x in range(1, nmonths+1):
        m += 1
        if m == 13:
            if len(mlist) != 0:
                d.append(gen_month_description(y, mlist))
            y += 1
            m = 1
            mlist=[]
        mlist.append(m)
    d.append(gen_month_description(y, mlist))
    
    return "MES AL COBRO " + ' Y '.join(d)

def get_description_months(description): 
    out = list()
    d = description.upper()
    d = d.replace('.', ',')
    d = d.replace('-',', ')
    d = re.sub(r'(AAL|LA|AL|SL)','AL',d)
    d = re.sub(r'(COBRO|CORO|COBERO)','COBRO',d)
    d = re.sub(r', SE LE HIZO.*','',d)
    d = re.sub(r', ABONO A.*','',d)
    d = re.sub(r', MAS 4.*','',d)
    d = re.sub(r' \(.*','',d)
    d = re.sub(r'.*SEGURIDAD MES','MES',d)
    d = d.replace('2033', '2023')
    
    #single month match
    m = re.match(r'^\s*M?ES\s+AL\s+COBRO\s*(\w+)\s+(\d+)\s*$', d)
    if m:
        out += get_months([m.group(1)], m.group(2))

    #range matches
    m = re.match(r'^\s*MES\s+AL\s+COBRO\s*(DE\s+)?(\w+)\s+A\s+(\w+)\s+(\d+)\s*$', d)
    if m:
        out += get_months([m.group(2), m.group(3)], m.group(4), get_range=True)
    
    m = re.match(r'^\s*MES\s+AL\s+COBRO\s+((\w+,\s+)*)(\w+)\s+Y\s+(\w+)\s+(\d+)\s*$', d)
    if m:
        months = [m.group(3), m.group(4)]
        if m.group(1):
            months = m.group(1).split(', ') + months
        out += get_months(months, m.group(5))
    
    #try splitting sentence
    if not out and re.search(r'Y', d):
        d_ = d.split('Y')
        for i in range(len(d_)-1):
            out += get_description_months('Y'.join(d_[0:i+1]))
            out += get_description_months('MES AL COBRO ' + 'Y'.join(d_[i+1:]))
    
    #try replacing , with Y
    if not out and re.search(r',', d):
        out += get_description_months(d.replace(',','Y'))
    
    out.sort(reverse=False)
    
    out_ = list()
    for x in out:
        if x not in out_:
            out_.append(x)
    return out_


def process_bill_excel():
    df = gendf_from_excel_table("data/facturas_quickbooks.xlsx",['Memo/Description'])
    df = df[1:][['Date','Num', 'Name', 'Memo/Description','Amount']].dropna(axis=0, how="all")
    
    #get mes
    df['mes'] = df['Memo/Description'].apply(get_description_months)
    
    #create original df copy
    df_allrows=df.copy(deep=True)
    df_allrows['Date']= pd.to_datetime(df_allrows['Date'], dayfirst=True)
    
    #group by name (client), adding date,num,description as lists
    for col in ['Date', 'Num', 'Memo/Description']:
        df[col] = df[col].apply(lambda x: [x])
    df = df.groupby(['Name']).sum()
    df.insert(0, 'customer', df.index)
    
    #add extrasummary columns
    df['ultima factura'] = df['Memo/Description'].apply(lambda x: x[-1] if x else np.nan)
    df['ultimo mes'] = df['mes'].apply(lambda x: x[-1] if x else np.nan)
    df['siguiente descripcion'] = df['ultimo mes'].apply(lambda x: gen_next_description(x,1) if isinstance(x,list) else np.nan)
    
    return [df_allrows, df] 

In [ ]:
print("Processing facturas")
#Load bills
[df_bills_allrows, df_bills] = process_bill_excel()
df_bills.to_excel('out/facturas.xlsx', index=False)

In [ ]:
############# REPORT ###########
figs = []

#Meses cancelados
df_bills_= df_bills.dropna(axis=0, subset="ultimo mes").explode('mes')
df_bills_['mes'] = df_bills_['mes'].apply(lambda x: str(x[0])+'-'+str(x[1]))
fig = px.histogram(df_bills_, x = "mes", text_auto=True, title = "Meses cancelados")
fig.update_layout(bargap=0.2)
figs.append(fig)

#Dinero que ingresa al mes
df_bills_ = df_bills_allrows
df_bills_["mes_factura"]=df_bills_['Date'].dt.strftime('%Y-%m')
df_bills_["tipo de pago"]=df_bills_['mes'].apply(lambda x: "cuota condominal" if x else "otros")
fig = px.histogram(df_bills_, x = "mes_factura", y = "Amount", color="tipo de pago",
                   text_auto=True, title = "Monto recaudado")
fig.update_layout(bargap=0.2)
figs.append(fig)

if not os.path.exists("out"):
    os.makedirs("out")
with open("out/facturas_reporte.html", "w") as fh:
    for fig in figs:
        fh.write(fig.to_html()) 

In [ ]:
print("Adding info from bancos")
#Process bancos if found to create file that will be imported to quickboks
if os.path.exists("out/bancos.xlsx"):
    df_bancos = gendf_from_excel_table("out/bancos.xlsx",['credito', 'casas'])
    df_bancos_valid =  df_bancos.loc[(df_bancos['customer'].notna()) & (df_bancos['num casas'].notna()) & (df_bancos['num meses'].notna()) &
              (df_bancos['num casas'] != 0) & (df_bancos['num meses'] != 0)]
    df_bancos_valid = df_bancos_valid.merge(df_bills, on=['customer'])
    df_bancos_valid['siguiente descripcion'] = df_bancos_valid.apply(lambda x: gen_next_description(x['ultimo mes'],x['num meses']), axis=1)
    df_bancos_valid.rename(columns={"descripcion": "detalle banco"}, inplace=True)
    #pd.to_datetime(df_bancos_valid['fecha'], dayfirst=True)
    colums = ['banco', 'referencia', 'detalle banco' ,'fecha', 'customer', 'credito', 'siguiente descripcion']
    df_bancos_valid[colums].to_excel('out/quickbooks_import.xlsx', index=False)

print("Task completed.")